# BING

In [ ]:
# !pip install requests
# !pip install bing-image-downloader

from bing_image_downloader import downloader
list = ["gas distribution meters on site images", "gas distribution valves on site images", "gas distribution regulators on site images", "gas distribution piping on site images"]
for i in range(len(list)):
    downloader.download(list[i], limit=50,  output_dir='bing', 
    adult_filter_off=True, force_replace=False, timeout=60)

# SELENIUM

In [ ]:
# !pip install shutil
# !pip install selenium

import os, shutil, urllib, time, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

main_folder = 'scrapped-data'
if not os.path.isdir(main_folder):
    os.makedirs(main_folder)
search = 'gas regulator on site images'
if os.path.isdir(main_folder + '/' + search):
    shutil.rmtree(main_folder + '/' + search)
if not os.path.isdir(main_folder + '/' + search):
    os.makedirs(main_folder + '/' + search)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('webdriver/chromedriver',chrome_options=chrome_options)
url ='https://www.google.com/imghp?hl=EN'
driver.get(url)
img = driver.find_element(By.XPATH,"//input[@class='gLFyf']")    
img.send_keys(search)
img.send_keys(Keys.ENTER)
src = []
SCROLL_PAUSE_TIME = 5
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    imgResults = driver.find_elements(By.XPATH,"//img[contains(@class,'Q4LuWd')]")
    for img in imgResults:
        if img.get_attribute('src')==None:
            continue

        src.append(img.get_attribute('src'))
        print(len(src))
        if len(src)==3000:
            break
            
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or len(src)==3000:
        try:
            driver.find_element(By.XPATH,"//div[@jsname='i3y3Ic']/input").click()    
        except:
            break
    last_height = new_height
    

for i in range(len(src)):
    try:
        urllib.request.urlretrieve(str(src[i]),f"{main_folder}/{search}/img{i}.jpg")
    except:
        continue

# BEAUTIFUL SOUP

In [ ]:
########################## downloads only 80 images at once #############################################################
# !pip install beautifulsoup4

import os
import requests
from bs4 import BeautifulSoup

google_image = "https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&"
user_agent = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
}
saved_folder = 'google-80'


def main():
    if not os.path.exists(saved_folder):
        os.mkdir(saved_folder)
    list = ["gas distribution meters on site images", "gas distribution valves on site images", "gas distribution regulators on site images", "gas distribution piping on site images"]
    for i in range(len(list)):
        os.mkdir(saved_folder + '/' + list[i])
        download_images(list[i])


def download_images(items):
#     data = input('What are you looking for? ')
    data = items
    n_images = 80
    print('searching...')
    search_url = google_image + 'q=' + data
    response = requests.get(search_url, headers=user_agent)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.findAll('img', {'class': 'rg_i Q4LuWd'})
    count = 1
    links = []
    for result in results:
        try:
            link = result['data-src']
            links.append(link)
            count += 1
            if(count > n_images):
                break
        except KeyError:
            continue
    print(f"Downloading {len(links)} images...")
    for i, link in enumerate(links):
        response = requests.get(link)
        image_name = saved_folder + '/' + data + '/' + data + str(i+1) + '.jpg'
        with open(os.path.join(image_name), 'wb') as fh:
            fh.write(response.content)

if __name__ == "__main__":
    main()

# GOOGLE LENS SEARCH

In [21]:
import os, urllib, time, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def scrap(file_path, t, scroll_rate, folder_path):
    try:
        #click no thanks sign in button
        iframe = driver.find_element("xpath", "//*[@id='gb']/div/div[3]/iframe")
        driver.switch_to.frame(iframe)
        driver.find_element(By.XPATH, "//div[@class ='QlyBfb']//button").click()
        driver.switch_to.default_content()
        print('Selected "no thanks sign in" option')
        time.sleep(t)

        #click search by image button
        cam_button = driver.find_element("xpath", "//div[@aria-label=\"Search by image\" and @role=\"button\"]")
        cam_button.click()
        print('Selected "search by image" option')
        time.sleep(t)

        # Find upload tab and find path
        upload_tab = driver.find_element("xpath", '//div[@class="BH9rn"]//div[@class="ZeVBtc"]//span')
        time.sleep(t)
        upload_btn = driver.find_element("name", "encoded_image")
        upload_btn.send_keys(file_path)
        print('Uploaded Image')
        time.sleep(t)
        
        #find image attributes
        src = []
        img = driver.find_elements(By.XPATH, "//div[@class='b57KQc']//div[@class='Me0cf ']//img")
        print('Scrolling to get Attributes...')

        for i in range(len(img)):
            driver.find_element("xpath", f'(//div[contains(@class, "Me0cf ")])[{i+1}]').location_once_scrolled_into_view
            time.sleep(scroll_rate)
            
        #get attribute from src
        for i in img:
            if i.get_attribute('src')==None:
                continue
            src.append(i.get_attribute('src'))

        #download images from url
        print('Downloading Images...')
        for j in range(len(src)):
            try:
                urllib.request.urlretrieve(str(src[j]),f"{folder_path}/img{j}.jpg")
            except:
                continue
        print('Images Downloaded:', j)
    
    #print error message
    except Exception as e:
        print(e)
        print('Trying again, Please wait...')

        #click search by image button
        cam_button = driver.find_element("xpath", "//div[@aria-label=\"Search by image\" and @role=\"button\"]")
        cam_button.click()
        print('Selected "search by image" option')
        time.sleep(t)

        # Find upload tab and find path
        upload_tab = driver.find_element("xpath", '//div[@class="BH9rn"]//div[@class="ZeVBtc"]//span')
        time.sleep(t)
        upload_btn = driver.find_element("name", "encoded_image")
        upload_btn.send_keys(file_path)
        print('Uploaded Image')
        time.sleep(t)
        
        src = []
        img = driver.find_elements(By.XPATH, "//div[@class='b57KQc']//div[@class='Me0cf ']//img")
        print('Scrolling to get Attributes...')

        for i in range(len(img)):
            driver.find_element("xpath", f'(//div[contains(@class, "Me0cf ")])[{i+1}]').location_once_scrolled_into_view
            time.sleep(scroll_rate)
            
        #get attribute from src
        for i in img:
            if i.get_attribute('src')==None:
                continue
            src.append(i.get_attribute('src'))

        #download images from url
        print('Downloading Images...')
        for j in range(len(src)):
            try:
                urllib.request.urlretrieve(str(src[j]),f"{folder_path}/img{j}.jpg")
            except:
                continue
        print('Images Downloaded:', j)

######################################################################################################################
#user input
file_path = r'C:\Users\seema1\Desktop\object-detection\images\a.jfif'    #path of the file to scrap images of
t = 10    #sleep time
scroll_rate = 0.5

try:
    #set directories
    main_folder = 'scrapped-data-lens'
    if not os.path.isdir(main_folder):
        os.mkdir(main_folder)

    a = file_path.split("\\")
    b = a[-1].split(".")
    pic = b[0]
    folder_path=main_folder + '/' + pic

    count=0
    while os.path.isdir(folder_path):
        count+=1
        folder_path=main_folder+'/'+pic+str(count)

    os.mkdir(folder_path)
    print('All directories set')

    #driver
#     options = webdriver.ChromeOptions()
#     options.add_argument('--headless')
#     options.add_argument('--no-sandbox')
#     options.add_argument('--disable-dev-shm-usage')
#     driver = webdriver.Chrome('webdriver/chromedriver', options=options)
#     url ='https://www.google.com/'
#     #url='https://images.google.com/'
#     driver.get(url)
    
    
    ser = Service(r"webdriver/chromedriver")
    op = webdriver.ChromeOptions()
    op.add_argument('--headless')
    op.add_argument('--no-sandbox')
    op.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=ser, options=op)
    url ='https://www.google.com/'
    driver.get(url)
       
    print('Opening Browser...')
    time.sleep(t)

    scrap(file_path, t, scroll_rate, folder_path)
    
except Exception as e:
    print(e)

All directories set
Opening Browser...
Selected "no thanks sign in" option
Selected "search by image" option
Uploaded Image
Scrolling to get Attributes...
Images Downloaded: 59


# GOOGLE IMAGE SEARCH

In [20]:
import os, shutil, urllib, time, requests, webbrowser, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

filepath = r'C:\Users\seema1\Desktop\object-detection\images\bb.jfif'    #path of the image to scrap images of
number_of_images = 50    #number of images to download
delay_in_seconds = 20

try:
    #get URL
    print('Getting URL...')
    searchUrl = 'https://www.google.com/searchbyimage/upload'
    multipart = {'encoded_image': (filepath, open(filepath, 'rb')), 'image_content': ''}
    response = requests.post(searchUrl, files=multipart, allow_redirects=False)
    fetchUrl = response.headers['Location']
    # webbrowser.open(fetchUrl)
    print('Got URL')
    
    #set directories
    print('Setting Directories...')
    main_folder = 'scrapped-data-images'
    if not os.path.isdir(main_folder):
        os.mkdir(main_folder)

    a = filepath.split("\\")
    b = a[-1].split(".")
    pic = b[0]
    folder_path=main_folder + '/' + pic

    count=0
    while os.path.isdir(folder_path):
        count+=1
        folder_path=main_folder+'/'+pic+str(count)

    os.mkdir(folder_path)
    print('All Directories Set')

#     #driver
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless')
#     chrome_options.add_argument('--no-sandbox')
#     chrome_options.add_argument('--disable-dev-shm-usage')
#     driver = webdriver.Chrome('webdriver/chromedriver',chrome_options=chrome_options)
#     driver.get(fetchUrl)
#     print('Opening Browser...')
    
    ser = Service(r"webdriver/chromedriver")
    op = webdriver.ChromeOptions()
    op.add_argument('--headless')
    op.add_argument('--no-sandbox')
    op.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=ser, options=op)
    driver.get(fetchUrl)
    print('Opening Browser...')

    time.sleep(delay_in_seconds)

    #open images page
    elem1 = driver.find_elements(By.XPATH,'//div[@class="hdtb-mitem"]//a')
    for i in range(len(elem1)):
        if elem1[i].text=="All":
            driver.find_elements("xpath", '//div[@class="hdtb-mitem"]//a')[i].click()
            break

    time.sleep(delay_in_seconds)
    elem2 = driver.find_elements(By.XPATH,'//div[@class="hdtb-mitem"]//a')
    for i in range(len(elem2)):
        if elem2[i].text=="Images":
            driver.find_elements("xpath", '//div[@class="hdtb-mitem"]//a')[i].click()
            break

    print('Opened Images Page. Downloading Images...')

    #get attributes of images
    src = []
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay_in_seconds)
        imgResults = driver.find_elements(By.XPATH,"//img[contains(@class,'Q4LuWd')]")
        for img in imgResults:
            if img.get_attribute('src')==None:
                continue

            src.append(img.get_attribute('src'))
            if len(src)==number_of_images:
                break

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height or len(src)==number_of_images:
            try:
                driver.find_element(By.XPATH,"//div[@jsname='i3y3Ic']/input").click()    
            except:
                break
        last_height = new_height

    #download images from web
    for i in range(len(src)):
        try:
            urllib.request.urlretrieve(str(src[i]),f"{folder_path}/img{i}.jpg")
        except:
            continue

    print('Images Downloaded: ', len(src))
    
#print error
except Exception as e:
    print('error', e)

Getting URL...
Got URL
Setting Directories...
All Directories Set
Opening Browser...
Opened Images Page. Downloading Images...
Images Downloaded:  50
